# Неделя 5

## Постановка задачи

### Построить прогноз на 6 часов вперед для всех концов истории. Нужно построить 6 независимых регрессионных моделей на каждый T+1

- Для каждой из шести задач прогнозирования сформируйте выборки. Возможные признаки:

  + идентификатор географической зоны — категориальный
  + год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
  + синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
  + значения прогнозов ARIMA
  + количество поездок из рассматриваемого района в моменты времени (аналог компоненты AR)
  + количество поездок из рассматриваемого района в моменты времени (аналог сезонной компоненты AR, суточной)
  + суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
  
- Разбейте каждую из шести выборок на три части:

  + обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
  + тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
  + итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016
  
- Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

  + допускала попарные взаимодействия между признаками
  + была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)
  
- Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по ранее указанному функционалу:

- Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

- Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

- Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

- Загрузите ноутбук в форму.

In [1]:
import os

%matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_columns", 500)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#Загружаем данные. Отбрасываем нерассматриваниые регионы
df = pd.read_csv('taxi_agg.csv', index_col=0)
# регионы отобрытнные на второй неделе (среднее к-во поездок в мае > 5)
regions = pd.read_csv('data_week2.csv', index_col=0).index 
df = df[df['bin_num'].isin(list(regions))]
df.index = pd.to_datetime(df.index)
df = df.set_index('Pickup_Hour')
df = df.pivot_table(values='trip_cnt', index=df.index, columns='bin_num', aggfunc='first')
df = df.fillna(0)

In [3]:
def create_fourier_fet(df, Kn):
    K = range(1, Kn+1)
    for sample_ks in K:
        sin = "sin_K_" + str(sample_ks)
        cos = "cos_K_" + str(sample_ks)
        df[sin] = np.sin(2*np.pi*sample_ks*np.arange(0, len(df))/168)
        df[cos] = np.cos(2*np.pi*sample_ks*np.arange(0, len(df))/168)
    return df

def extract_weekday(df):
    
    names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    df.index = pd.to_datetime(df.index)
    df = df.join(pd.get_dummies(df.index.get_level_values(0).weekday_name) \
                .set_index(df.index).reindex(columns=names))
    
    return df

def extract_hour(df):
    
    df.index = pd.to_datetime(df.index)
    df = df.join(pd.get_dummies(df.index.get_level_values(0).hour, prefix = 'hour') \
                 .set_index(df.index))
    return df

def extract_month(df):
    
    df.index = pd.to_datetime(df.index)
    df = df.join(pd.get_dummies(df.index.get_level_values(0).month, prefix = 'month') \
                 .set_index(df.index))
    return df

def extract_ar(df, reg, per, step = 1):
    new_df = pd.DataFrame(index = df.index)
    for i in range(0, per):
        new_column = 'ar_' + str(i) + '_' + str(step)
        new_series = df[reg].shift(i*step)
        new_df.loc[:, new_column] = new_series
    return new_df

def extract_sum(df, reg, per):
    new_df = pd.DataFrame(index = df.index)
    new_column = 'sum_' + str(per)
    new_series = df[reg].rolling(min_periods=per, window=per).sum()
    new_df.loc[:, new_column] = new_series
    return new_df

def extract_mean(df, reg, per):
    new_df = pd.DataFrame(index = df.index)
    new_column = 'mean_' + str(per)
    new_series =  df[reg].rolling(min_periods=per, window=per).mean()
    new_df.loc[:, new_column] = new_series
    return new_df

In [4]:
#extract universal features 
df_reg = pd.DataFrame(index = df.index)
df_reg = create_fourier_fet(df_reg, 15) #Фурье
df_reg = extract_weekday(df_reg) #Недели
df_reg = extract_hour(df_reg) #Часы
df_reg = extract_month(df_reg) #Месяцы

In [5]:
Xy = pd.DataFrame()
for each_region in tqdm(regions):
    
    df_reg['region'] = each_region
    
    #df for variable features
    df_prel = pd.DataFrame(index = df.index)
    # количество поездок из рассматриваемого района в моменты времени
    df_prel = df_prel.join(extract_ar(df, each_region, 24, step = 1)) # 24 часа с шагом 1 час
    df_prel = df_prel.join(extract_ar(df, each_region, 6, step = 24)) # шаг три дня (сезонность)
    
    df_prel = df_prel.join(extract_sum(df, each_region, 6)) #cумма за последние 6 часов
    df_prel = df_prel.join(extract_sum(df, each_region, 12)) #cумма за последние 12 часов
    df_prel = df_prel.join(extract_sum(df, each_region, 24)) #cумма за последние 24 часа
    df_prel = df_prel.join(extract_sum(df, each_region, 168)) #cумма за последнюю неделю
    
    df_prel = df_prel.join(extract_mean(df, each_region, 6)) #среднее за последние 6 часов
    df_prel = df_prel.join(extract_mean(df, each_region, 12)) #среднее за последние 12 часов
    df_prel = df_prel.join(extract_mean(df, each_region, 24)) #среднее за последние 24 часа
    df_prel = df_prel.join(extract_mean(df, each_region, 168)) #среднее за последнюю неделю
    
    df_prel['snow 23-01-2016'] = 0
    df_prel.loc['2016-01-23':'2016-01-24', 'snow 23-01-2016'] = 1
    
    df_prel = df_prel.join(df_reg)
    df_prel['region'] = each_region
    # true values 6 hours ahead
    for i in range(1, 7):
        df_prel['y'+str(i)] = df[each_region].shift(-i) 
    
    
    Xy = Xy.append(df_prel, ignore_index = False)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [6]:
#Делим выборку
#обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
#тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
#итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016
Xy.dropna(axis = 0, inplace = True)
y_train = Xy.loc[:'2016-04-30 23:00:00', Xy.columns[-7:]]
y_test = Xy.loc['2016-05-01 00:00:00' : '2016-05-31 23:00:00', Xy.columns[-7:]]
y_fin = Xy.loc['2016-06-01 00:00:00' :, Xy.columns[-7:]]

X_train = Xy.loc[:'2016-04-30 23:00:00', Xy.columns[:-7]]
X_test = Xy.loc['2016-05-01 00:00:00' : '2016-05-31 23:00:00', Xy.columns[:-7]]
X_fin = Xy.loc['2016-06-01 00:00:00' :, Xy.columns[:-7]]

In [ ]:
#Choosing n_estimators
file_r = 'week5_estimators.txt'
if os.path.exists(file_r) == False:
    with open(file_r, 'w') as f:
        f.write('l_estimators, r_sq, mae\n')

l_estimators = [10, 25, 50, 75, 100, 125, 150, 300, 500]
r_sq = []
mae = []

for each_est in tqdm(l_estimators):
    RF = RandomForestRegressor(n_estimators = each_est, random_state = 0, n_jobs = -1)
    RF.fit(X_train, y_train['y1'])
    y_hat = RF.predict(X_test)
    
    mae.append(mean_absolute_error(y_test['y1'], y_hat))
    r_sq.append(r2_score(y_test['y1'], y_hat))
    
    with open('week5_estimators.txt', 'a') as f:
        f.write(str(each_est) + ',' + str(r_sq) + ',' + str(mae) + '\n')

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [17]:
#Обучаем линейную регрессию
model_ens = {}
for each_hour in tqdm(range(1, 7)):
    model_nm = 'model_rf_{}'.format(str(each_hour))
    y_hour = 'y{}'.format(str(each_hour))
    model_ens[model_nm] = RandomForestRegressor(random_state = 0, n_jobs = -1)
    model_ens[model_nm].fit(X_train, y_train[y_hour])

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [19]:
y_hat_1 = model_ens['model_ln_reg_1'].predict(X_test)
y_hat_2 = model_ens['model_ln_reg_2'].predict(X_test)
y_hat_3 = model_ens['model_ln_reg_3'].predict(X_test)
y_hat_4 = model_ens['model_ln_reg_4'].predict(X_test)
y_hat_5 = model_ens['model_ln_reg_5'].predict(X_test)
y_hat_6 = model_ens['model_ln_reg_6'].predict(X_test)

In [28]:
print(sum(abs(y_test['y1'] - y_hat_1))/len(y_hat_1))
print(sum(abs(y_test['y2'] - y_hat_2))/len(y_hat_2))
print(sum(abs(y_test['y3'] - y_hat_3))/len(y_hat_3))
print(sum(abs(y_test['y4'] - y_hat_4))/len(y_hat_4))
print(sum(abs(y_test['y5'] - y_hat_5))/len(y_hat_5))
print(sum(abs(y_test['y6'] - y_hat_6))/len(y_hat_6))

16.01815302344706
18.582298094063308
19.397161661867482
19.918985328397476
20.28750034279499
20.42389140271537


In [29]:
(sum(abs(y_test['y1'] - y_hat_1)) + sum(abs(y_test['y2'] - y_hat_2)) + 
    sum(abs(y_test['y3'] - y_hat_3)) + sum(abs(y_test['y4'] - y_hat_4)) + 
    sum(abs(y_test['y5'] - y_hat_5)) + sum(abs(y_test['y6'] - y_hat_6))) / (len(y_hat_1)*6)

19.104664975547614

In [8]:
df_reg.columns.append(df.columns)

Int64Index([1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131,
            ...
            1630, 1684, 1733, 1734, 1783, 2068, 2069, 2118, 2119, 2168],
           dtype='int64', length=102)